In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")


/opt/homebrew/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

# Load your CSV
df = pd.read_csv("awesomeData.csv")  # Replace with actual path

# Access the DNA and gRNA columns
target_dna_list = df["target_dna"].tolist()
grna_list = df["grna_seq"].tolist()

# Optionally inspect
print("Example DNA:", target_dna_list[0])
print("Example gRNA:", grna_list[0])


Example DNA: CGGCGCTGGTGCCCAGGACGAGGATGGAGATT
Example gRNA: CGGCGCTGGTGCCCAGGACGAGGATGGAGATT


In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")
model.eval()  # set to inference mode

# Tokenize target DNA
dna_tokens = tokenizer(target_dna_list, return_tensors="pt", padding=True, truncation=True)
dna_emb = model(**dna_tokens, output_hidden_states=True).hidden_states[-1]  # last layer
dna_mean = dna_emb.mean(dim=1)  # mean pool over sequence length

# Tokenize gRNAs
grna_tokens = tokenizer(grna_list, return_tensors="pt", padding=True, truncation=True)
grna_emb = model(**grna_tokens, output_hidden_states=True).hidden_states[-1]
grna_mean = grna_emb.mean(dim=1)

# One-to-one similarity
cos_sim = F.cosine_similarity(grna_mean, dna_mean)  # shape: (N,)
df["cosine_similarity"] = cos_sim.detach().numpy()

print(df[["target_dna", "grna_seq", "cosine_similarity"]].head())
df.to_csv("grna_dna_similarity_output.csv", index=False)

: 